## Split Audio
Use MediaConvert to split audio into segments

In [1]:
import boto3
client = boto3.client('mediaconvert')

In [19]:
response = client.create_job(
    Queue="arn:aws:mediaconvert:us-east-1:374169031176:queues/Default",
    Role="arn:aws:iam::374169031176:role/service-role/MediaConvert_Default_Role",
    BillingTagsSource="JOB",
    AccelerationSettings={
        "Mode": "DISABLED"
    },
    StatusUpdateInterval="SECONDS_60",
    Priority=0,
    Settings={
    "TimecodeConfig": {
      "Source": "ZEROBASED"
    },
    "OutputGroups": [
      {
        "CustomName": "_chop",
        "Name": "Apple HLS",
        "Outputs": [
          {
            "ContainerSettings": {
              "Container": "M3U8",
              "M3u8Settings": {}
            },
            "AudioDescriptions": [
              {
                "AudioSourceName": "Audio Selector 1",
                "CodecSettings": {
                  "Codec": "AAC",
                  "AacSettings": {
                    "Bitrate": 28000,
                    "CodingMode": "CODING_MODE_1_0",
                    "SampleRate": 16000,
                    "Specification": "MPEG4"
                  }
                }
              }
            ],
            "OutputSettings": {
              "HlsSettings": {}
            },
            "NameModifier": "_chop"
          }
        ],
        "OutputGroupSettings": {
          "Type": "HLS_GROUP_SETTINGS",
          "HlsGroupSettings": {
            "SegmentLength": 60,
            "Destination": "s3://stt-test-batch-transform/output/",
            "DestinationSettings": {
              "S3Settings": {
                "StorageClass": "STANDARD"
              }
            },
            "MinSegmentLength": 0
          }
        }
      }
    ],
    "FollowSource": 1,
    "Inputs": [
      {
        "AudioSelectors": {
          "Audio Selector 1": {
            "DefaultSelection": "DEFAULT"
          }
        },
        "TimecodeSource": "ZEROBASED",
        "FileInput": "s3://stt-test-batch-transform/input/STT_NS_3500.wav"
      }
    ]
  } 
)

In [22]:
get_job_response = client.get_job(
    Id=response['Job']['Id']
)

## Check Job status

get_job_response['Job']['Status']

## Delete the .m3u8 files 
.m3u8 files gets generated along with the .aac files we need. To pass the folder as input to SageMaker Batch Transform we have to delete non audio files.

In [24]:
import boto3

s3_resource = boto3.resource('s3')
bucket = s3_resource.Bucket('stt-test-batch-transform')
objects = bucket.objects.filter(Prefix = 'output/')

objects_to_delete = [{'Key': o.key} for o in objects if o.key.endswith('.m3u8')]

if len(objects_to_delete):
    s3_resource.meta.client.delete_objects(Bucket='stt-test-batch-transform', Delete={'Objects': objects_to_delete})

In [25]:
import boto3

sagemaker_client = boto3.client('sagemaker')

In [42]:
response = sagemaker_client.create_transform_job(
    TransformJobName='test-7-boto3',
    ModelName='huggingface-pytorch-inference-2024-02-28-06-57-20-020',
    MaxConcurrentTransforms=1,
    TransformInput={
        'DataSource': {
            'S3DataSource': {
                'S3DataType': 'S3Prefix',
                'S3Uri': 's3://stt-test-batch-transform/output/'
            }
        },
        'ContentType': 'audio/x-audio',
        'CompressionType': 'None',
        'SplitType': 'None'
    },
    TransformOutput={
        'S3OutputPath': 's3://stt-test-batch-transform/inference',
        'Accept': 'application/json',
        'AssembleWith': 'None'
    },
    TransformResources={
        'InstanceType': 'ml.g4dn.xlarge',
        'InstanceCount': 1
    }
)

### Check Job status

In [52]:
response = sagemaker_client.describe_transform_job(
    TransformJobName='test-7-boto3'
)
response['TransformJobStatus']

'Completed'

In [53]:
import boto3

s3_resource = boto3.resource('s3')
bucket = s3_resource.Bucket('stt-test-batch-transform')
objects = bucket.objects.filter(Prefix = 'inference/')

# objects_to_delete = [{'Key': o.key} for o in objects if o.key.endswith('.m3u8')]

# if len(objects_to_delete):
#     s3_resource.meta.client.delete_objects(Bucket='stt-test-batch-transform', Delete={'Objects': objects_to_delete})

In [54]:
objects

s3.Bucket.objectsCollection(s3.Bucket(name='stt-test-batch-transform'), s3.ObjectSummary)

In [55]:
def list_files_in_folder(bucket_name, folder_name):
    # Create a Boto3 client for S3
    s3_client = boto3.client('s3')
 
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=folder_name)
 
 
    if 'Contents' in response:
    
        for obj in response['Contents']:
            print(obj['Key'])
    else:
        print("Folder is empty.")

In [57]:
bucket_name='stt-test-batch-transform'
folder_name='inference/'
list_files_in_folder(bucket_name, folder_name)

inference/
inference/STT_NS_3500_chop_00001.aac.out
inference/STT_NS_3500_chop_00002.aac.out
inference/STT_NS_3500_chop_00003.aac.out
inference/STT_NS_3500_chop_00004.aac.out
inference/STT_NS_3500_chop_00005.aac.out
inference/STT_NS_3500_chop_00006.aac.out
inference/STT_NS_3500_chop_00007.aac.out
inference/STT_NS_3500_chop_00008.aac.out
inference/STT_NS_3500_chop_00009.aac.out
inference/STT_NS_3500_chop_00010.aac.out
inference/STT_NS_3500_chop_00011.aac.out
inference/STT_NS_3500_chop_00012.aac.out
inference/STT_NS_3500_chop_00013.aac.out
inference/STT_NS_3500_chop_00014.aac.out
inference/STT_NS_3500_chop_00015.aac.out
inference/STT_NS_3500_chop_00016.aac.out
inference/STT_NS_3500_chop_00017.aac.out
inference/STT_NS_3500_chop_00018.aac.out
inference/STT_NS_3500_chop_00019.aac.out
inference/STT_NS_3500_chop_00020.aac.out
inference/STT_NS_3500_chop_00021.aac.out
inference/STT_NS_3500_chop_00022.aac.out
inference/STT_NS_3500_chop_00023.aac.out
inference/STT_NS_3500_chop_00024.aac.out
infer

In [77]:
import json
text=""
for obj in list(bucket.objects.filter(Prefix = 'inference/'))[1:]:
    key = obj.key
    body = json.loads(obj.get()['Body'].read().decode('utf-8'))
    print(key)
    print(body['text'])
    text += body['text']

inference/STT_NS_3500_chop_00001.aac.out
དེ་རིང་གི་་བརྗོར་གཞི་དེ་ག་རེ་འདྲ་བོ་ཆགས་ཀི་འདུག་ལབ་ན། ཨེ་ནས་ང་རང་ད་ད་ལྟ་བར་དུ་ད་ཅིག་སོ་སོ་སོ་དེ་བསྡད་ཀི་ཡོད་རཱ་ད་ག་རེ་ལབ་དགོས་རེད། ཆུང་ཆུང་ནས་བྱས་ནས་ད་ལྟ་བར་དུ་ཀ་ཅི་ད་དུས་ཡུན་འདྲ་བོ་ལབ་ན་རེད་རཱ། ་ཨོ་དེའི་སྐོར་ལ་ཅིག་བཤད་ཡ་ཡིན་རཱ། གསལ་བོ་བྱས་ནས་ཨེ་ནས་ཆུང་ཆུང་ནས་བྱས་ནས་ད་ལྟ་བར་དུ་ཅིག་ང་རང་གི་མི་ཚེ་ཆུང་ཆུང་ཞིག་མཐོང་ཡོད་རཱ་ད། ཐུང་ཐུང་ཅིག་མཐོང་ཡོད་རཱ། ཨོ་དེའི་སྐབར་ལ་གཅིག་བཤད་དགོས་བསམ་སོང་། ད་ཆུང་ཆུང་སྐབས་སུ་འདྲ་བོ་ཆ་བཞག་བ་ཡིན་ན། ཨེ་ནས་དང་ཤོས་དང་ཤོས་ད་ང་ག་རེ་ལབ་དགོས་རེད། དཔེ་བསྟོན་སློབ་གྲྭར་འགྲོ་གི་ཡོད་རེད། ཆུང་ཆུང་སྐབས་སུ་ཨེ་ནས་ཨོ་དེ་འགྲོ་དུས་ཙམ་པ་ཨེ་ནས་དཔེ་སྟོན་སློབ་གྲྭ་ལ་ང་ཚོ་དབྱིན་ཇི་བསླབ་ཡ་གི་ལུགས་སྲོལ་ཡོད་མ་རེད་རཱ། ང་ཚོ་དུས་ཚོད་སྐབས་སུ། ང་ཕལ་ཆེར་འཛིན་གྲྭ་དང་པོ་དང་པོ་ཡང་མིན་ན་གཉིས་པ་བར་དུ་འདྲ་བོ་ཅིག་ཡོད་བ།ང་དཔེ་བསྟོན་སློབ་གྲྭར་འགྲོ་མྱོང་ཡོད་རཱ། ཨེ་ནས་ཨོ་དེ་ཡིན་དུས་ཙམ་པ་ལ་ཨེ་ནས་གཅིག་ག་རེ་ལབ་དགོས་རེད། བོད་ཡིག་གི་གཞི་ཚང་མ་ཡོད་རཱ་ ད་གཞི་རྩ་ཚང་མ་ད་ང་ཚོ་ལ་ཆུང་ཆུང་ནས་བསླབ་ཚར་རེད་ཡོད་བ། འཛིན་གྲྭ་དང་པོ་ཟེ་ནས་ཁོ་རང་བསླབ་ཚར་སོང་རཱ། ཨེ་ནས་མཇུག་ནས་སློབ་གྲྭར་འག

In [78]:
text

'དེ་རིང་གི་་བརྗོར་གཞི་དེ་ག་རེ་འདྲ་བོ་ཆགས་ཀི་འདུག་ལབ་ན། ཨེ་ནས་ང་རང་ད་ད་ལྟ་བར་དུ་ད་ཅིག་སོ་སོ་སོ་དེ་བསྡད་ཀི་ཡོད་རཱ་ད་ག་རེ་ལབ་དགོས་རེད། ཆུང་ཆུང་ནས་བྱས་ནས་ད་ལྟ་བར་དུ་ཀ་ཅི་ད་དུས་ཡུན་འདྲ་བོ་ལབ་ན་རེད་རཱ། ་ཨོ་དེའི་སྐོར་ལ་ཅིག་བཤད་ཡ་ཡིན་རཱ། གསལ་བོ་བྱས་ནས་ཨེ་ནས་ཆུང་ཆུང་ནས་བྱས་ནས་ད་ལྟ་བར་དུ་ཅིག་ང་རང་གི་མི་ཚེ་ཆུང་ཆུང་ཞིག་མཐོང་ཡོད་རཱ་ད། ཐུང་ཐུང་ཅིག་མཐོང་ཡོད་རཱ། ཨོ་དེའི་སྐབར་ལ་གཅིག་བཤད་དགོས་བསམ་སོང་། ད་ཆུང་ཆུང་སྐབས་སུ་འདྲ་བོ་ཆ་བཞག་བ་ཡིན་ན། ཨེ་ནས་དང་ཤོས་དང་ཤོས་ད་ང་ག་རེ་ལབ་དགོས་རེད། དཔེ་བསྟོན་སློབ་གྲྭར་འགྲོ་གི་ཡོད་རེད། ཆུང་ཆུང་སྐབས་སུ་ཨེ་ནས་ཨོ་དེ་འགྲོ་དུས་ཙམ་པ་ཨེ་ནས་དཔེ་སྟོན་སློབ་གྲྭ་ལ་ང་ཚོ་དབྱིན་ཇི་བསླབ་ཡ་གི་ལུགས་སྲོལ་ཡོད་མ་རེད་རཱ། ང་ཚོ་དུས་ཚོད་སྐབས་སུ། ང་ཕལ་ཆེར་འཛིན་གྲྭ་དང་པོ་དང་པོ་ཡང་མིན་ན་གཉིས་པ་བར་དུ་འདྲ་བོ་ཅིག་ཡོད་བ།ང་དཔེ་བསྟོན་སློབ་གྲྭར་འགྲོ་མྱོང་ཡོད་རཱ། ཨེ་ནས་ཨོ་དེ་ཡིན་དུས་ཙམ་པ་ལ་ཨེ་ནས་གཅིག་ག་རེ་ལབ་དགོས་རེད། བོད་ཡིག་གི་གཞི་ཚང་མ་ཡོད་རཱ་ ད་གཞི་རྩ་ཚང་མ་ད་ང་ཚོ་ལ་ཆུང་ཆུང་ནས་བསླབ་ཚར་རེད་ཡོད་བ། འཛིན་གྲྭ་དང་པོ་ཟེ་ནས་ཁོ་རང་བསླབ་ཚར་སོང་རཱ། ཨེ་ནས་མཇུག་ནས་སློབ་གྲྭར་འགྲོ་དུས་ཙམ།པ་རཱ།ད་ང་མཇུག་ནས་ད་བོད་ཁྱིམ་ཅོ